The can sends and receives packets, with this code we simulate the motor driver and a real-time response. 
Below is the explanation of the code . 
Jupyter fails to import the can library, try it on python directly

In [1]:
import can
import time
import threading

# === Variabili simulazione ===
gear = 0
mode = 0
throttle = 0
max_speed = 2000
input_current = 100
fault_flags = 0x0000  # tutti OK

# === Funzione di simulazione driver ===
def ezoutboard_driver(bus):
    global gear, mode, throttle, max_speed, input_current, fault_flags

    while True:
        msg = bus.recv(timeout=1)
        if msg:
            # === GESTIONE MESSAGGI ===

            # 0x1A530106 – Imposta modalità CAN e marcia
            if msg.arbitration_id == 0x1A530106:
                mode = msg.data[0]
                gear = msg.data[1]
                print(f"[DRIVER] Modalità CAN: {mode}, Marcia: {gear}")

            # 0x1A530100 – Imposta acceleratore (PWM)
            elif msg.arbitration_id == 0x1A530100:
                throttle = (msg.data[0] << 8) | msg.data[1]
                print(f"[DRIVER] Throttle (PWM): {throttle}/65535")

            # 0x1A530101 – Parametri: max speed, current
            elif msg.arbitration_id == 0x1A530101:
                max_speed = (msg.data[0] << 8) | msg.data[1]
                input_current = (msg.data[2] << 8) | msg.data[3]
                print(f"[DRIVER] Max speed: {max_speed} RPM, Input current: {input_current * 0.1:.1f} A")

            # === RISPOSTA: stato operativo (0x1A530601) ===
            speed_rpm = int(throttle / 65535 * max_speed)
            voltage = 4800  # 480.0 V
            current = int(input_current * throttle / 65535)

            status_msg = can.Message(
                arbitration_id=0x1A530601,
                data=[
                    (speed_rpm >> 8) & 0xFF, speed_rpm & 0xFF,
                    0x07, 0xD0,  # gear ratio fittizio (2000)
                    (voltage >> 8) & 0xFF, voltage & 0xFF,
                    (current >> 8) & 0xFF, current & 0xFF
                ],
                is_extended_id=True
            )
            bus.send(status_msg)

            # === RISPOSTA: stato fault (0x1A530600) ===
            fault_msg = can.Message(
                arbitration_id=0x1A530600,
                data=[
                    (fault_flags >> 8) & 0xFF, fault_flags & 0xFF,
                    0x00,  # gear status, brake, cruise, ecc.
                    0x00,  # reverse, TCS, ecc.
                    int(throttle / 65535 * 200),  # throttle % (0–200)
                    0x00,  # reserved
                    60,    # controller temp
                    55     # motor temp
                ],
                is_extended_id=True
            )
            bus.send(fault_msg)

            time.sleep(0.2)


# === Simulazione invio comandi ===
def controller_interface(bus):
    while True:
        print("\n[CONTROLLER] Inserisci comando:")
        print("1 - Imposta modalità CAN + Marcia")
        print("2 - Imposta Throttle (PWM)")
        print("3 - Imposta parametri (Velocità max / Corrente max)")
        print("4 - Esci")
        choice = input("> ")

        if choice == "1":
            mode = int(input("Modalità (0=Display, 1=CAN): "))
            gear = int(input("Marcia (0=VOID, 1=Eco, 2=Normal, 3=Sport): "))
            msg = can.Message(arbitration_id=0x1A530106, data=[mode, gear] + [0]*6, is_extended_id=True)
            bus.send(msg)

        elif choice == "2":
            throttle_pct = float(input("Throttle % (0–100): "))
            val = int(throttle_pct / 100 * 65535)
            msg = can.Message(arbitration_id=0x1A530100,
                              data=[(val >> 8) & 0xFF, val & 0xFF] + [0]*6, is_extended_id=True)
            bus.send(msg)

        elif choice == "3":
            rpm = int(input("Velocità max (RPM): "))
            current = int(float(input("Corrente max (A): ")) * 10)
            msg = can.Message(
                arbitration_id=0x1A530101,
                data=[
                    (rpm >> 8) & 0xFF, rpm & 0xFF,
                    (current >> 8) & 0xFF, current & 0xFF,
                    4, 0x00,   # pole pairs = 4
                    0x00, 0x00
                ],
                is_extended_id=True
            )
            bus.send(msg)

        elif choice == "4":
            break

        time.sleep(0.2)


# === Main ===
with can.Bus(interface='virtual') as bus:
    driver_thread = threading.Thread(target=ezoutboard_driver, args=(bus,))
    driver_thread.start()

    controller_interface(bus)

    driver_thread.join()


<class 'ModuleNotFoundError'>: No module named 'can'

We're simulating an electric motor controlled via CAN bus.
We're doing it using Python, to test and learn — without needing any real hardware.



In [ ]:
import can
import time
import threading


In [ ]:
gear = 0
mode = 0
throttle = 0
max_speed = 2000
input_current = 100
fault_flags = 0x0000  # all OK


These are the variables that represent the motor’s state.

gear → current gear (e.g. 1 = Eco, 2 = Balanced, 3 = Sport)

mode → control mode (0 = Manual/Display, 1 = CAN controlled)

throttle → how much you're pressing the gas (range 0–65535)

max_speed → max RPM the motor can reach

input_current → max current the motor can draw (in 0.1 A units)

fault_flags → error flags (0x0000 means no errors)



In [ ]:
def ezoutboard_driver(bus):


This function simulates the motor. It listens for CAN messages, processes them, and sends back motor status.

bus is the virtual CAN channel used for communication.



In [ ]:
global gear, mode, throttle, max_speed, input_current, fault_flags

This line says: “Hey, I’m going to use the variables defined outside of this function.”

In [ ]:
        if msg.arbitration_id == 0x1A530106:
            mode = msg.data[0]
            gear = msg.data[1]
            print(f"[DRIVER] CAN Mode: {mode}, Gear: {gear}")


This message tells the motor:

how it should be controlled (manual or CAN)

which gear to use

msg.data[0] → control mode (0 = Display, 1 = CAN)

msg.data[1] → gear (1 = Eco, 2 = Balanced, 3 = Sport)

In [ ]:
        elif msg.arbitration_id == 0x1A530100:
            throttle = (msg.data[0] << 8) | msg.data[1]
            print(f"[DRIVER] Throttle (PWM): {throttle}/65535")


This sets the throttle value, using two bytes.

Throttle range is 0–65535 (16-bit number).

32768 = about 50% throttle

In [ ]:
        elif msg.arbitration_id == 0x1A530101:
            max_speed = (msg.data[0] << 8) | msg.data[1]
            input_current = (msg.data[2] << 8) | msg.data[3]
            print(f"[DRIVER] Max speed: {max_speed} RPM, Input current: {input_current * 0.1:.1f} A")


The controller sends a message to update:

Max speed (RPM)

Max current (as tenths of an amp — 100 = 10.0 A)

In [ ]:
        speed_rpm = int(throttle / 65535 * max_speed)
        voltage = 4800  # 480.0 V
        current = int(input_current * throttle / 65535)


Calculate motor state and send back status.
speed_rpm → motor RPM based on throttle

voltage → fixed simulated battery voltage (you can make it dynamic)

current → current draw, proportional to throttle

In [ ]:
        status_msg = can.Message(
            arbitration_id=0x1A530601,
            data=[
                (speed_rpm >> 8) & 0xFF, speed_rpm & 0xFF,
                0x07, 0xD0,
                (voltage >> 8) & 0xFF, voltage & 0xFF,
                (current >> 8) & 0xFF, current & 0xFF
            ],
            is_extended_id=True
        )
        bus.send(status_msg)


Create status message (ID 0x1A530601)
This sends a status update from the motor, including:

Bytes 0–1: RPM (2 bytes)

Bytes 2–3: gear ratio (fake value of 2000)

Bytes 4–5: battery voltage

Bytes 6–7: motor current

All values are split into high and low bytes.

In [ ]:
        fault_msg = can.Message(
            arbitration_id=0x1A530600,
            data=[
                (fault_flags >> 8) & 0xFF, fault_flags & 0xFF,
                0x00, 0x00,
                int(throttle / 65535 * 200),
                0x00,
                60,  # controller temp
                55   # motor temp
            ],
            is_extended_id=True
        )
        bus.send(fault_msg)


Send fault status (ID 0x1A530600)
This sends another message with:

Fault flags (errors — 0 means no errors)

Throttle level as %

Temperatures for controller and motor